In [7]:
import pymatgen as pmg
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

import lammps

In [6]:
structure = CifParser('../data/structure/MgO.cif').get_structures()[0]
spga = SpacegroupAnalyzer(structure)
conventional_structure = spga.get_conventional_standard_structure()

In [10]:
lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(conventional_structure)

In [11]:
# add potential